# Data Gathering and Pre-processing

Import the required libraries...

In [1]:
import csv
import pickle
import os

## Gathering

NFL play by play data found [here](https://github.com/nflverse/nflverse-data/releases/tag/pbp). In this cell we will convert the csv files to pkl files, so they're easier to work with. This part may take a while, as the csv files are pretty big.

In [2]:
# loop through the play by play csv files
for i in range(1999, 2023):
    # open the csv file for the year (i is the year)
    with open(f'./pbp_csv/play_by_play_{i}.csv', 'r') as f:
        reader = csv.reader(f)
        # make pbp_pkl folder before running, if it doesn't exist
        try:
            os.makedirs(f'./pbp_pkl')
        except:
            pass
        # create the pickle file for the year, and dump the csv data into it. Store it in pbp_pkl folder
        pickle.dump(list(reader), open(f'./pbp_pkl/play_by_play_{i}.pkl', 'wb'))

Next, gather data on team ratings by season from 